In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
i = 0

for sqrt_value in generator:
    i = i+1
    sum = sum + sqrt_value
    print(f'{i} : {sqrt_value}, sum: {sum}')
    if (i == 5): print(f'Answer 1: {sum}')
    if (i == 13): print(f'Answer 2: {sqrt_value}')

1 : 1.0, sum: 1.0
2 : 1.4142135623730951, sum: 2.414213562373095
3 : 1.7320508075688772, sum: 4.146264369941973
4 : 2.0, sum: 6.146264369941973
5 : 2.23606797749979, sum: 8.382332347441762
Answer 1: 8.382332347441762
6 : 2.449489742783178, sum: 10.83182209022494
7 : 2.6457513110645907, sum: 13.47757340128953
8 : 2.8284271247461903, sum: 16.30600052603572
9 : 3.0, sum: 19.30600052603572
10 : 3.1622776601683795, sum: 22.4682781862041
11 : 3.3166247903554, sum: 25.7849029765595
12 : 3.4641016151377544, sum: 29.249004591697254
13 : 3.605551275463989, sum: 32.854555867161245
Answer 2: 3.605551275463989


In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [19]:
# load with append
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# load people_1
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# I skipped calculating the total age of people_1 dataset
# proceed to append

# load people_2
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

In [20]:
# analyze appended
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people data:")

people_appended = conn.sql("SELECT * FROM people").df()
display(people_appended)

people_appended_total_age = conn.sql("SELECT SUM(age) FROM people").df()

print('Answer 3:')
display(people_appended_total_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708293631.330174,QCwyyWa0fIG5aA,None
1,2,Person_2,27,City_A,1708293631.330174,68eN68Z7S1AxSQ,None
2,3,Person_3,28,City_A,1708293631.330174,4JW6+esC2Axqgw,None
3,4,Person_4,29,City_A,1708293631.330174,iMFupynkzq0CMg,None
4,5,Person_5,30,City_A,1708293631.330174,csA3fUqW8UMPNA,None
5,3,Person_3,33,City_B,1708293632.4482765,8z/0+VDVavIOcw,Job_3
6,4,Person_4,34,City_B,1708293632.4482765,cNQgZULJ7xTK5w,Job_4
7,5,Person_5,35,City_B,1708293632.4482765,f4D6wWhtjfUV1A,Job_5
8,6,Person_6,36,City_B,1708293632.4482765,iFsOYzZK2qz06Q,Job_6
9,7,Person_7,37,City_B,1708293632.4482765,Eq7l4vgoFUb5jg,Job_7


Answer 3:


,sum(age)
0,353.0


In [15]:
# load with merge

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# load people_1
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# load people_2
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge",
                                        primary_key="id")

In [18]:
# analyze merged

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people data:")

people_merged = conn.sql("SELECT * FROM people ORDER BY id").df()
display(people_merged)

people_merged_total_age = conn.sql("SELECT SUM(age) FROM people").df()

print('Answer 4:')
display(people_merged_total_age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708293206.2266705,wSmdZz4zKPJHyg,None
1,2,Person_2,27,City_A,1708293206.2266705,uJBUebThy8Cg4g,None
2,3,Person_3,33,City_B,1708293207.1557863,LzLPzZOnxQPuhw,Job_3
3,4,Person_4,34,City_B,1708293207.1557863,M9Zv3jgzHc3Pcg,Job_4
4,5,Person_5,35,City_B,1708293207.1557863,GUg4WW9FSsE3DA,Job_5
5,6,Person_6,36,City_B,1708293207.1557863,6U++AvINrBxdzA,Job_6
6,7,Person_7,37,City_B,1708293207.1557863,llUbHhVYyyXbOQ,Job_7
7,8,Person_8,38,City_B,1708293207.1557863,VISUzn9VwJjtIQ,Job_8


Answer 4:


,sum(age)
0,266.0
